In [2]:
'''
utility:

main
play_mode
make_board
print_board
to_score
pause
clear
get_key_press
get_piece
place_piece
place_random - input board, output: board with random tile spawn
board_full
have_lost
possible_moves
to_move_list
move
swipe
transform_board - input: board, direction, output: board
automate
find_max
countEmpty
'''

import random
import time
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from functools import partial
from tabulate import tabulate

import os
#Importing non-standard libraries
try:
    import getch
except ImportError:
    getch = None
try:
    import termcolor
except ImportError:
    termcolor = None
try:
    from tkinter import *
    #Assigned to None because import was successful but $DISPLAY may not be set
    gui, GUI_runnable, = None, None
except ImportError:
    gui, GUI_runnable, = False, False

pausetime = 0 #time to pause between AI moves (for visibility)
board_size = 4
N = 4 #board_size
move_table = [0]*65535

def main():
    
    #Creating my new 4x4 board
    board = make_board(N)
    
    #Getting the game started with two pieces on the board
    place_random(board)
    place_random(board)

    print_board(board)
    
    print("Press p to play the game using arrow keys");
    print("Press a to automate gameplay using one of the in-built programs");
    print("Press s to get statistics for one of the in-built programs");
    print("Press q to quit");
    
    while True:

        #Gets the key pressed and stores it in the key variable
        key = ord(input("Choose game mode: "))
        
        #play case ('p')
        if key==112:
            play_mode(board)
            
        #automate case ('a')
        if key==97:
            automate(board)
            
        #statistics case ('s')
        if key==115:
            getStatistics()
        
        #Quit case ('q')
        else:
            print("Game Finished!");
            quit()
            break
            
def play_mode(board):
    """
    Game mode that allows the user to play using arrow keys
    """
    while True:
            
        key = get_key_press()

        if key == 65:
            swipe(board, 0)

        elif key == 66:
            swipe(board, 1)

        elif key == 67:
            swipe(board, 2)

        elif key == 68:
            swipe(board, 3)
            
        print_board(board)
            
        #Check to see if I've lost at the end of the game or not
        if have_lost(board):
            maxval = max(max(row) for row in board)
            score = to_score(board)
            print("Game over. Final score %d; highest tile %d." % (score, maxval))
            print("You lost! Would you like to play again? (y/n)");
            if (input() == 'y'):
                main();
            else:
                quit()
                
def make_board(N):
    """
    Utility function that returns a new N x N empty board (empty spaces represented by '*')
    Arg N: integer - board dimensions - must be greater than or equal to 1
    """
    assert N >= 1, "Invalid board dimension";
    assert type(N) == int, "N must be an integer";
    #return np.array([0 for x in range(N) for x in range(N)]); #represent board as N*N length array
    return np.array(['0' for x in range(N) for x in range(N)]); #represent board as 16 hexadecimal entries

def print_board(board):
    """
    Utility function that prints out the state of the board
    Arg board: board - the board you want to print
    """
    colors = {
        '0': None,
        '2': 'red',
        '4': 'green',
        '8': 'yellow',
        '16': 'blue',
        '32': 'magenta',
        '64': 'cyan',
        '128': 'grey',
        '256': 'white',
        '512': 'green',
        '1024': 'red',
        '2048': 'blue',
        '4096': 'magenta'
    };

    score = to_score(board)
    
    print("Your score: "+str(score));

    vertical_edge = "";
    for i in range(N+2):
        vertical_edge += "-\t";
    print(vertical_edge);
    
    for y in range(N):
        row = "";
        for x in range(N):
            cell = get_piece(x,y,board) #find the right index on the board
            if cell==0: row+= '0'
            else: row += str(2**cell)
            row += "\t";
        print("|\t" + row + "|");
        if y is not N-1: print("")
    print(vertical_edge);
        
def _to_score(c):
    #input: cell in binary
    #output: score from cell
    if c <= 1:
        return 0
    return (c-1) * (2**c)

def to_score(board):
    #input: board
    #output: board score
    #need to adjust for generated 4s
    cell_scores = [_to_score(get_piece(x,y,board)) for x in range(N) for y in range(N)]
    return(sum(cell_scores))
        
def pause(seconds):
    """
    Utility function that pauses for the given amount of time
    Arg seconds: a float or integer - number of seconds to pause for
    """
    time.sleep(seconds);
    
def clear():
    """Utility function that clears the terminal GUI's screen - takes no arguments"""
    try:
        try:
            #For Macs and Linux
            os.system('clear');
        except:
            #For Windows            REPORTED BUG: Sometimes does not work on 64 bit Windows
            os.system('cls');
    except:
        #If nothing else works, a hacky, non optimal solution
        for i in range(50): print("")
            
def get_key_press():
    """Utility function that gets which key was pressed and translates it into its character ascii value - takes no arguments"""
    return ord(getch.getch());

def get_piece(x, y, board):
    """
    Utility function that returns the int power of 2 of the piece at a given (x,y) coordinate on the given board
    Returns the piece if the request was valid and None if the request was not valid
    Arg x: integer - x coordinate
    Arg y: integer - y coordinate
    Arg board: board - the board you wish to get the piece from
    """

    #Ensure that x and y are both integers (use assert)
    assert type(x) == type(y) == int, "Coordinates must be integers"

    #size constraint on x and y
    #Checking that the (x,y) coordinates given are valid for the N x N board
    if x >= N or y >= N or x < 0 or y < 0:
        return None

    #Getting the piece on the board
    piece = int(board[4*y+x],16) #converting to int from hexadecimal
    return piece

def place_piece(piece, x, y, board):
    """
    Utility function that places the piece at a given (x,y) coordinate on the given board if possible
    Will overwrite the current value at (x,y), no matter what that piece is
    Returns True if the piece is placed successfully and False otherwise
    Arg piece: string - represents a piece on the board (as an integer power of 2)
    Arg x: integer - x coordinate
    Arg y: integer - y coordinate
    Arg board: board - the board you wish to place the piece on
    """
    
    #Ensure that x and y are both integers (use assert)
    assert type(x) == type(y) == int, "Coordinates must be integers"


    #Checking that the (x,y) coordinates given are valid for the board
    if x >= N or y >= N or x < 0 or y < 0:
        return False

    #Placing the piece on the board as a hexadecimal string
    board[4*y+x] = hex(piece)[2:]
    return True

def place_random(board):
    """
    Helper function which is necessary for the game to go to the next move
    Returns True if a piece is placed and False if the board is full
    Places a 2 (90%) or 4 (10%) randomly on the board in an empty space
    Arg board: board - the board you wish to place the piece on
    """
    
    #Check if the board is full and return False if it is
    if board_full(board): return False;

    #random.random() generates a random decimal between [0, 1) ... Multiplying by 100 generates a number between [0, 100)
    generated = random.random() * 100;

    #Assign to_place according to my generated random number

    if generated < 90:                              
        to_place = 1

    else:
        to_place = 2

    #Variable keeps track of whether a randomly generated empty spot has been found yet
    found = False

    while not found:
        #Generate a random (x,y) coordinate that we can try to put our new value in at
        random_y = int(random.random() * N)
        random_x = int(random.random() * N)

        #If the randomly generated coordinates are empty, we have found a spot to place our random piece
        found = get_piece(random_x, random_y, board) == 0

    #Place the piece at the randomly generated (x,y) coordinate
    place_piece(to_place,random_x,random_y,board)

    return True

def board_full(board):
    """
    Utility function that returns True if the given board is full and False otherwise
    Arg board: board - the board you want to check
    """
    for piece in board:
        if piece == '0':  return False; #check string equality here

    return True;

def have_lost(board):
    """
    Helper function which checks at the end of each turn if the game has been lost
    Returns True if the board is full and no possible turns exist and False otherwise
    Arg board: board - the board you wish to check for a losing state
    """
    moves = possible_moves(board)
    if len(moves)==0: #if no moves possible
        return True
    else:
        return False
    
def possible_moves(board):
    """
    Utility function that, given a board, will return a list of the moves that are possible
    Arg board: board - the board you wish to check for possible moves
    """
    N = board_size
    possible = [False, False, False, False]; #which moves are possible (up,left, bottom, right)

    #Check every (x,y) position on the board to see which moves are possible
    for y in range(N):
        for x in range(N):
            piece_at_xy = get_piece(x, y, board); #check each piece
            if piece_at_xy == None: #not a valid piece, go to next iteration (logically should never happen)
                continue;
            elif piece_at_xy == 0: #we want to find where existing pieces can move to
                continue;

            else:
                if piece_at_xy == get_piece(x+1, y, board) or piece_at_xy == get_piece(x-1, y, board): 
                #has an equal piece to the right or left
                    possible[3] = True;
                    possible[1] = True;
                if piece_at_xy == get_piece(x, y+1, board) or piece_at_xy == get_piece(x, y-1, board): 
                #has an equal piece down or up
                    possible[2] = True;
                    possible[0] = True;

                if get_piece(x+1, y, board)== 0: #has an empty spot to the right
                    possible[3] = True;
                if get_piece(x-1, y, board)== 0: #has an empty spot to the left
                    possible[1] = True;
                if get_piece(x, y+1, board)== 0: #has an empty spot below
                    possible[2] = True;
                if get_piece(x, y-1, board)== 0: #has an empty spot above
                    possible[0] = True;
    
    moves = to_move_list(possible)
    return moves;

def to_move_list(possible):
    """helper function for possible_moves function
    returns a list of the possible moves"""
    moves = []
    if possible[0]:
        moves.append(0)
    if possible[1]:
        moves.append(1)
    if possible[2]:
        moves.append(2)
    if possible[3]:
        moves.append(3)
    return moves

def move(row):
    """
    Utility function that moves the inputted row towards the left
    Returns row after movement
    """
    #don't need for each direction - do 1 and use symmetry
    
    #has been replaced by more efficient look up table (see movequick function)
    
    n = len(row)
    
    mergeable = [True]*n
    
    for i in range(len(row)):
        row[i]=int(row[i],16) #convert from hexadecimal to int
        #possible to avoid this conversion and reconversion and do all below in hex?
        
    for i in range(n): #going through each element in row
        
        while True:
            
            val = row[i] #current tile
            last = None
            
            if val==0: #empty tile
                break
            
            if i>0:
                last=row[i-1] #last tile

            if last==None: #if nowhere to move, go to next tile
                break

            elif last==0: #move to blank tile
                row[i-1] = val
                row[i] = 0
                i = i-1

            elif mergeable[i] and (val==last): 
                row[i]=0 #move this tile out
                row[i-1] = val + 1 #double the last tile
                mergeable[i-1] = False
                i = i-1

            else: #unequal tile to the left or already merged
                break
                
    for i in range(len(row)):
        row[i]=hex(row[i])[2:] #convert from int to hexadecimal
        
    return row

def swipe(board, direction):
    """
    inputs a board and direction
    outputs board after swiping in the given direction
    """
    #tracks whether any action was taken
    action_taken = False

    #valid_direction = (direction == 0 or direction == 1 or direction == 2 or direction == 3);
    #assert valid_direction, "Invalid direction passed in";  #Logical debug case
    
    original = copy.deepcopy(board) #store original board
    
    #transform to a symmetric LEFT swipe case
    board = transform_board(board, direction)
    #optimise this too: make sure indexing/transformation is not a bottleneck
    
    #swipe for each row in the transformed board (in the same direction)
    #use pre-generated look up table to do this
    for i in range(4):
        row = board[4*i:4*i+4]
        newrow = movequick(row) #how to do this without using the bottleneck functions?
        board[4*i:4*i+4] = newrow

    #transform back to original
    board = transform_board(board, direction)
 
    #if we did something this move
    if board is not original:
        place_random(board)
    
    return board
        
def transform_board(board, direction):
    """
    transforms board to the symmetric case for any other direction
    (i.e., moving the input board in direction is equivalent to moving the outputted board left)
    """
    #change to use indexing rather than calcualtionso
    if direction==1: 
        return board
    
    if direction==3:
        indices = [3,2,1,0,7,6,5,4,11,10,9,8,15,14,13,12]
        return board[indices]
    
    if direction == 0:
        indices = [0,4,8,12,1,5,9,13,2,6,10,14,3,7,11,15]
        return board[indices]
    
    if direction == 2:
        indices = [12,8,4,0,13,9,5,1,14,10,6,2,15,11,7,3]
        return board[indices]
    
def automate(board):
    """runs the game automatically using the chosen AI"""

    key = int(input("Which AI would you like to run? "));
    num = input("How many games would you like the AI to play? ")
    
    start =  time.clock()
    scores = []
    max_tiles = []
    
    for i in range(int(num)):
        
        while True:

            global pausetime
            #pause(pausetime); #for better visibility of watching AI play

            #possible moves
            moves = possible_moves(board)

            #check if lost
            if len(moves)==0: #if no move is possible
                    scores.append(to_score(board)) #add this score to our score list
                    current_max = findMax(board)[0][0]
                    max_tiles.append(current_max)
                    print_board(board) #for debugging
                    
                    if(i!=int(num)-1): #unless we're at the last iteration
                        #Creating a new 4x4 board
                        board = make_board(board_size)
                        #Getting the game started with two pieces on the board
                        place_random(board)
                        place_random(board)
                    break

            #if only one move possible then just move
            if len(moves)==1:
                board = swipe(board, moves[0])

            else:
                #run the chosen AI
                #get moves from the AIs program
                #implement them
                #randomise AI (input '1')
                    
                if key == 1:
                    move = randomise(board)

                #sequential AI (input '2')
                elif key == 2:
                    move = sequential(board)

                #random alternating AI (input '3')
                elif key == 3:
                    move = random_altern(board)

                #always up else left (input '4')
                elif key == 4:
                    move = alwaysup(board)

                #AI5 (input '5')
                elif key == 5:
                    move = AI5(board)

                #AI6 (input '6')
                elif key == 6:
                    move = AI6(board)

                #AI5 (input '7')
                elif key == 7:
                    move = AI7(board)

                #AI8 (input '8')
                elif key == 8:
                    move = AI8(board)

                #AI9 (input '9')
                elif key == 9:
                    move = AI9(board)

                #AI10 (input '10')
                elif key == 10:
                    move = AI10(board)

                #AI11 (input '11')
                elif key == 11:
                    move = AI11(board)

                #AI12
                elif key == 12:
                    move = AI12(board)
                    
                #AI13
                elif key == 13:
                    move = AI13(board)
                    
                board = swipe(board,move) #make move based on AI input
                
    end = time.clock()
    print("AI "+str(key))
    print("Time taken for "+str(num)+" tries: "+str(end-start))
    
    #compute score statistics
    print("The average score is: "+str(np.mean(scores)));
    print("The minimum score is: "+str(np.min(scores)));
    print("The maximum score is: "+str(np.max(scores)));
    print("The standard deviation of scores is: "+str(np.std(scores)));
    #plt.hist(scores,bins=20)
    #plt.show()
    #plt.close()
    log_scores = np.log(scores)
    print("The average log score is: "+str(np.mean(log_scores)));
    print("The standard deviation of log scores is: "+str(np.std(log_scores)));
    #plt.hist(log_scores,bins=20)
    #plt.show()
    #plt.close()
    print(max_tiles)
    print("The average max tile is: "+str(np.mean(max_tiles)));
    print("The minimum max tile is: "+str(np.min(max_tiles)));
    print("The maximum max tile is: "+str(np.max(max_tiles)));
    print("The standard deviation of the max tile is: "+str(np.std(max_tiles)));
    #plt.hist(max_tiles,bins=10)
    #plt.show()
    #plt.close()
    main()
                
def findMax(board):
    """input: board
    output: vector with maximum tile and it's locations [[max, x, y],[max, x, y],...]
    finds the maximum tile and all its locations (since not necessarily unique)
    Helper function for AI
    """
    max_val = 0;
    max_vec = [];

    #find the biggest tile and store its locations
    for i in range(N):
        for j in range(N):
            current = get_piece(i, j, board);
            if current > max_val:
                max_val = current; #update our max
                max_vec = []; #empty out locations of previous max
            if not current<max_val: #if we found a new location for our previous max
                max_vec.append([2**current, i, j]) #store max val and it's location in an array
    return max_vec;

def countEmpty(board):
    """input: board
    output: int with number of empty tiles on the board
    Helper function for AI
    """
    count = 0
    #find the empty tiles
    for i in range(N):
        for j in range(N):
            current = get_piece(i, j, board);
            if current == 0:
                count += 1
    return count;

def MonteCarloEval(board,n,move,d=0):
    """after making move on board, plays randomly n times for d moves(till the end if d=0) and returns score"""

    current_board = copy.deepcopy(board)

    move_score = 0

    for j in range(n): #run n times

        board = copy.deepcopy(current_board) #reset board state
        movecount = 0

        board = swipe(board, move) #make the move

        while not have_lost(board): #play game to end randomly

            movecount += 1
            if(d>0):
                if movecount>d:
                    break

            moves_now = possible_moves(board)
            move_now = random.choice(moves_now) #move randomly
            board = swipe(board, move_now)

        move_score += to_score(board) #record score
        
    return move_score

In [4]:
#board as string of length 16
#each tile a hex

def convertRowtoBits(row):
    """input: a row of 4 hexadecimals
    output: row encoded in bits"""
    #encode row as a 16 bit unsigned integer
    num = np.uint16(int(row[0],16)*4096 + int(row[1],16)*256 +16*int(row[2],16)+int(row[3],16))
    #num = np.uint16(int(row,16))
    return num

def convertBitstoRow(bits):
    """input: a row of 4 integers encoded as a 16 bit unsigned int
    output: row as hexadecimals"""
    #can eliminate this function if look up table stores hex and not int
    row=[0,0,0,0]
    bits = np.binary_repr(bits) #converts to binary
    while(len(bits)<16):
        bits = '0'+bits
    for i in range(N):
        row[i]=hex(int(bits[4*i:4*i+4],2))[2:] #store as decimal integer converting from binary
    return row

def movequick(row):
    """uses a lookup tabe to quickly figure out what a row will look like after it moves
    input: row as a list of decimal powers
    outputs: a list of decimal tile powers"""
    start = convertRowtoBits(row)
    return(move_table[start])

def generate_move_table():
    for i in range(65535): #each row (encoded as 16 bits unsigned integer)
        #each entry of table as a string of length 4 with 4 hex
        row = convertBitstoRow(i)
        newrow = move(row) #generate ith entry of table
        move_table[i] = newrow #convert to bit representation

#generate_move_table() #generate the table
#print(move_table)


In [5]:
def randomise(board):
    """1: instead of the player playing the game, randomly choose one of the 4 moves each turn"""
    moves = possible_moves(board)
    return random.choice(moves)

def alwaysup(board):
    """4: automates gameplay with each move up, else left, else down, else right"""
    #repeated computation (automate already calculates possible moves)
    moves=possible_moves(board)
    return moves[0]

    
def AI7(board):
    """greedy algorithm that chooses that automates gampeplay by choosing the move that most increases the score everyt turn"""
    N = board_size
    
    current_board = copy.deepcopy(board)
    current_score = to_score(board)
    delta_score = [0,0,0,0] #change in score with each move
    
    moves = possible_moves(board)

    for i in range(len(moves)): #for each move possible
        new_board = swipe(current_board, moves[i]) #make move
        delta_score[i] = to_score(new_board) - current_score #record change in score
        
        best = delta_score.index(max(delta_score))
        return moves[best]
    
def AI10(board):
    """greedy algorithm that chooses that automates gampeplay by choosing the move that maximises empty tiles on the board"""
    current_board = copy.deepcopy(board) #save current_board
    moves = possible_moves(board)
    empty = [0,0,0,0]
    
    for i in range(len(moves)): #calculate empty tiles for each move
        swipe(current_board, moves[i])
        empty[i] = countEmpty(board)
        
        best = empty.index(max(empty)) #if multiple max, chooses the first one
        return moves[best] #make the move that gives us the most empty tiles at the end
    
def AI11(board):
    """for each move: play game to m moves randomly n times, choose the starting move that leads to the best score """

    n = 10 #number of times to play the game to end for each

    original_board = copy.deepcopy(board) #save board state now
    final_score = [0,0,0,0]
    moves = possible_moves(board)

    for i in range(len(moves)): #for each possible move
        final_score[i] = MonteCarloEval(board,n,moves[i]) #record score
        board = copy.deepcopy(original_board)
    
    #final_scores = np.array(final_score)
    #print(final_scores/n)
    best = final_score.index(max(final_score)) #if multiple max, chooses the first one
    return moves[best] #make the move with the best final score on average
    
def AI12(board, d=3,n=10):
    """for each move: play game to m moves randomly n times, choose the starting move that leads to the best score """

    #d = 3 #depth of search
    #n = 5 #number of times to play the game to end for each

    original_board = copy.deepcopy(board) #save board state now
    final_score = [0,0,0,0]
    moves = possible_moves(board)

    for i in range(len(moves)): #for each possible move
        final_score[i] = MonteCarloEval(original_board,n,moves[i],d) #record score
        board = copy.deepcopy(original_board)

    #final_scores = np.array(final_score)
    #print(final_scores/n)
    best = final_score.index(max(final_score)) #if multiple max, chooses the first one
    return moves[best] #make the move with the best final score on average

def AI13(board,d=3,t=5):
    """tries to keep the maximum tiles in corners (preferred move)
    uses montecarlo evaluation to choose between moves on equal preference
    built for a 4x4 board
    if unique max: (exactly like AI5)
        if in corner: stay in corner (hard preferences)
        if on edge: try and move to closest corner (weighted preferences)
        if in center: move towards nearest corner (weighted preferences)
    if not unique max: 
        calculate preferences for each max and choose probabilistically
    """
    
    original_board = copy.deepcopy(board) #save board state now
    final_score = [0,0,0,0]
    moves = possible_moves(board)
    preferred = [] #list of most preferred moves
    N = board_size
    max_vec = findMax(board) #vectors with all maxima

    for current_max in max_vec: #for each max
        x = current_max[1] #location of max
        y = current_max[2]
        #corner case
        #give high priority to corners (i.e. a corner move preference has infinitely more weight than an edge preference)
        #for corner, max must be (0,0), (n-1,0), (0,n-1) or (n-1,n-1)
        if (x == 0 or x == N-1) and (y == 0 or y == N-1):
            if y==0 and 0 in moves: #if we're in the top corner and moving up is also possible
                preferred.append(0)
            if x==0 and 1 in moves: #left edge and left move possible
                preferred.append(1)
            if y==N-1 and 2 in moves: #if we're in the bottom corner and moving down is also possible
                preferred.append(2) 
            if x==N-1 and 3 in moves: #right edge and right move possible
                preferred.append(3)
        
    preferred = list(set(preferred)) #remove duplicates

    #once done building preference and weights for all maxima
    if len(preferred)>0: #if we have a move preference
        if len(preferred)==1:
            return preferred[0]
        
        else:
            moves = preferred #choose from preferred moves
    
    for i in range(len(moves)): #for each possible move
        final_score[i] = MonteCarloEval(original_board,t,moves[i],d) #record score
        board = copy.deepcopy(original_board)

    final_scores = np.array(final_score)
    print(final_scores/t)
    best = final_score.index(max(final_score)) #if multiple max, chooses the first one
    return moves[best] #make the move with the best final score on average

In [8]:
def getStatisticsMonteCarlo():
    """runs the game repeatedly for a chosen AI"""
    
    start = time.clock()
    #Creating a new 4x4 board
    board = make_board(board_size)
    #Getting the game started with two pieces on the board
    place_random(board)
    place_random(board)
    
    num = input("How many trials would you like to run for each set of parameters? ")
    
    t_vec = [10]
    
    d_vec = [20,25,30,35]
    score_result = [[0 for d in d_vec] for t in t_vec]
    log_result = [[0 for d in d_vec] for t in t_vec]
    max_mean = [[0 for d in d_vec] for t in t_vec]
    max_result = [[0 for d in d_vec] for t in t_vec]
    log_max = [[0 for d in d_vec] for t in t_vec]
    
    for t in range(len(t_vec)):
        for d in range(len(d_vec)):
            scores = []
            max_tiles = []
            for i in range(int(num)):
                while True:
                    #possible moves
                    moves = possible_moves(board)
                    #check if lost
                    if len(moves)==0: #if no move is possible
                        scores.append(to_score(board)) #add this score to our score list
                        current_max = findMax(board)[0][0]
                        max_tiles.append(current_max)
    
                        if not ((i==int(num)-1)and(t_vec[t]==t_vec[-1])and(d_vec[d]==d_vec[-1])): #unless we're at the last iteration
                            #Creating a new 4x4 board
                            board = make_board(board_size)
                            #Getting the game started with two pieces on the board
                            place_random(board)
                            place_random(board)

                        break

                        #if only one move possible then just move
                    if len(moves)==1:
                        board = swipe(board, moves[0])

                    else:
                        move = AI12(board, d=d_vec[d], n=t_vec[t])
                        board = swipe(board,move) #make move based on AI input

            print("Depth of search: "+str(d_vec[d]))
            print(scores)
            print(max_tiles)
            print(" ")
            score_result[t][d] = np.mean(scores)
            log_result[t][d] = np.mean(np.log(scores))
            max_mean[t][d] = np.mean(max_tiles)
            max_result[t][d] = np.amax(max_tiles)
            log_max[t][d] = np.mean(np.log2(max_tiles))#mean of log base 2 of max tiles
            
    end = time.clock()
    print("Time taken: "+str(end-start))
    print("Number of trials: "+str(num))
    print("Number of threads: "+str(t))
    print(score_result)
    print(log_result)
    print(max_mean)
    print(max_result)
    print(log_max)

In [11]:
import numpy as np
    
def getStatistics():
    """runs the game repeatedly for a chosen AI"""
    
    start = time.clock()
    #Creating a new 4x4 board
    board = make_board(board_size)
    #Getting the game started with two pieces on the board
    place_random(board)
    place_random(board)
    
    num = int(input("How many trials would you like to run? "))
    
    score_result = np.zeros(num)
    max_result = np.zeros(num)
    
    for i in range(num):

        while True:
            #possible moves
            moves = possible_moves(board)
            
            #check if lost
            if len(moves)==0: #if no move is possible
                score_result[i]=to_score(board) #add this score to our score list
                max_result[i] = findMax(board)[0][0]

                #Creating a new 4x4 board
                board = make_board(board_size)
                #Getting the game started with two pieces on the board
                place_random(board)
                place_random(board)

                break

            #if only one move possible then just move
            elif len(moves)==1:
                board = swipe(board, moves[0])

            else:
                move = randomise(board) #CHOOSE ALGORITHM HERE
                board = swipe(board,move) #make move based on AI input
            
    print(score_result)
    print(max_result)
    np.savetxt('data.csv', (score_result, max_result), delimiter=',')
    end = time.clock()
    print("Time taken: "+str(end-start))
    print("Number of trials: "+str(num))
    return [score_result, max_result]

In [12]:
getStatistics()

How many trials would you like to run? 1000
[ 1156.  1056.  1380.  2176.   608.  1188.   748.  1156.  1168.  1360.
  1108.  1092.  2432.   508.  1548.  1232.  1472.  1216.  1760.  1156.
   660.  1276.  1844.  1232.   648.  1120.   488.   816.   628.   864.
  1300.  2528.  1420.  1328.  1292.  1328.   672.   484.  2356.   876.
  1308.  1368.   844.   692.  1644.  1264.   320.  1456.  1300.   736.
  1984.  1344.  1280.  1504.  1364.   928.  2492.   444.  1500.  1308.
  1376.  2228.   600.   656.  1144.   836.   688.   700.  1088.   412.
   732.  1348.   892.  1100.   660.  2444.  1384.  1468.   436.  1188.
  2256.  2212.  2384.  1512.  1500.   736.   900.  1640.   596.  1584.
  1552.   860.   764.  1172.   752.  1256.  1296.  1424.   564.  1472.
  1804.  2436.  2452.  1364.   252.   812.   464.  2416.  1332.  1616.
  1292.   828.   652.  1376.   764.  1412.   448.   860.  1144.   608.
   608.   248.   732.  1456.  1452.   960.  1376.  2460.  2232.  1008.
  1428.  1348.  1568.   400.   93

In [13]:
main()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
%matplotlib inline
score_result = [9578.3999999999996, 13681.68, 14729.52, 15966.16, 16520.240000000002, 15924.4, 13813.200000000001, 14887.440000000001, 15708.799999999999, 14394.639999999999]
log_result = [9.0751308186345589, 9.4782194498445822, 9.4373511672513999, 9.5824947443755075, 9.6172638636497734, 9.5701231957766986, 9.4043872947321407, 9.4689681324329484, 9.529205038922127, 9.466592045280958]
max_mean = [742.39999999999998, 972.79999999999995, 1029.1199999999999, 1116.1600000000001, 1152.0, 1100.8, 998.39999999999998, 1075.2, 1059.8399999999999, 977.91999999999996]
max_result = [1024, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]
log_max = [9.4000000000000004, 9.8599999999999994, 9.8000000000000007, 9.9800000000000004, 10.02, 9.9600000000000009, 9.7799999999999994, 9.8800000000000008, 9.8800000000000008, 9.8000000000000007]
d_vec = [1,2,3,4,5,6,7,8,9,10]
plt.plot(d_vec, score_result)
plt.show()

In [ ]:
%matplotlib inline
data=score_table
plt.imshow(data, aspect='auto',extent=[0,34,20,1])
plt.savefig('py.png')
plt.colorbar()
plt.show()

In [16]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='nadhirajk', api_key='ngZYUrpgsnjZRPdXbDRS')

# Add data
month = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
         'August', 'September', 'October', 'November', 'December']
n_vec = [1,2,3,4,5,7,10,15,20]

#5 games for each depth at each thread number
d0 = [4182.4, 9420.8, 7489.6, 14296 , 9760, 10311.2, 16155.2, 28117.6, 27336]

# Create and style traces
trace0 = go.Scatter(
    x = n_vec,
    y = d0,
    name = 'Infinite',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

data = [trace0]

# Edit the layout
layout = dict(title = 'Average High and Low Temperatures in New York',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Temperature (degrees F)'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

In [1]:
import plotly.figure_factory as ff

#find a way to plot bars for each max tile (4,8,16,32,64,128,256,512,...) with one bar each for each algorithm (differnete colours)
#and size of bar depended on number of times we got that max tile

# Add histogram data: 1st entry score, 2nd entry max tiles
#x1 = np.loadtxt('random.csv', delimiter=',')
#x2 = np.loadtxt('sequential.csv', delimiter=',') 
#x3 = np.loadtxt('randomaltern.csv', delimiter=',') 
#x4 = np.loadtxt('alwaysup.csv', delimiter=',') 

# Group data together
hist_data = [x1[1], x2[1], x3[1], x4[1]]

group_labels = ['Random', 'Sequential', 'Randomly Alternate', 'Always Up']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)

# Plot!
#py.iplot(fig, filename='Blind Algorithms Distribution of Maximum Tiles')

NameError: name 'x1' is not defined

    100% |████████████████████████████████| 1.1MB 500kB/s eta 0:00:01
  Running setup.py bdist_wheel for plotly ... done
  Stored in directory: /Users/Neel/Library/Caches/pip/wheels/11/0e/f6/22208411ddcbfa81a89ae341490c5d12b1df5be988aad494b6
Successfully built plotly
